<a href="https://colab.research.google.com/github/gagan3012/keytotext/blob/master/Notebooks/modeltexttokey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://gitlab.com/shimorina/webnlg-dataset.git
!git clone https://github.com/Yale-LILY/dart.git
!pip install datasets
!pip install transformers
!pip install sentencepiece

Cloning into 'webnlg-dataset'...
remote: Enumerating objects: 5106, done.
remote: Total 5106 (delta 0), reused 0 (delta 0), pack-reused 5106
Receiving objects: 100% (5106/5106), 26.08 MiB | 21.84 MiB/s, done.
Resolving deltas: 100% (4008/4008), done.
Checking out files: 100% (1425/1425), done.
     |████████████████████████████████| 204kB 19.3MB/s 
     |████████████████████████████████| 245kB 45.9MB/s 
     |████████████████████████████████| 112kB 55.8MB/s 
     |████████████████████████████████| 2.1MB 17.4MB/s 
     |████████████████████████████████| 3.3MB 55.8MB/s 
     |████████████████████████████████| 901kB 52.8MB/s 
     |████████████████████████████████| 1.2MB 19.0MB/s 


In [ ]:
import pandas as pd
import torch
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [ ]:
import glob

files =[]
dirc = ['/content/webnlg-dataset/release_v2.1/xml/train/**/*.xml',
        '/content/webnlg-dataset/release_v3.0/en/train/**/*.xml',
        '/content/webnlg-dataset/webnlg_challenge_2017/train/**/*.xml']
for dir in dirc:
  file = glob.glob("{}".format(dir), recursive=True)
  files.append(file)


f = files[0] + files[1] + files[2]
f

['/content/webnlg-dataset/release_v2.1/xml/train/7triples/7triples_University_train_release.xml',
 '/content/webnlg-dataset/release_v2.1/xml/train/7triples/7triples_Monument_train_release.xml',
 '/content/webnlg-dataset/release_v2.1/xml/train/7triples/7triples_Astronaut_train_release.xml',
 '/content/webnlg-dataset/release_v2.1/xml/train/6triples/6triples_University_train_release.xml',
 '/content/webnlg-dataset/release_v2.1/xml/train/6triples/6triples_Astronaut_train_release.xml',
 '/content/webnlg-dataset/release_v2.1/xml/train/6triples/6triples_Monument_train_release.xml',
 '/content/webnlg-dataset/release_v2.1/xml/train/4triples/4triples_Building_train_release.xml',
 '/content/webnlg-dataset/release_v2.1/xml/train/4triples/4triples_WrittenWork_train_release.xml',
 '/content/webnlg-dataset/release_v2.1/xml/train/4triples/4triples_Artist_train_release.xml',
 '/content/webnlg-dataset/release_v2.1/xml/train/4triples/4triples_CelestialBody_train_release.xml',
 '/content/webnlg-dataset/re

In [ ]:
import glob
import os
import re
import xml.etree.ElementTree as ET

files = glob.glob("/content/dart/data/v1.1.1/dart-v1.1.1-full-train.xml")
triple_re=re.compile('(\d)triples')
data_dct={}
for file in f:
    tree = ET.parse(file)
    root = tree.getroot()
    triples_num=int(triple_re.findall(file)[0])
    for sub_root in root:
        for ss_root in sub_root:
            strutured_master=[]
            unstructured=[]
            for entry in ss_root:
                unstructured.append(entry.text)
                strutured=[triple.text for triple in entry]
                strutured_master.extend(strutured)
            unstructured=[i for i in unstructured if i.replace('\n','').strip()!='' ]
            strutured_master=strutured_master[-triples_num:]
            strutured_master_str=(' && ').join(strutured_master)
            data_dct[strutured_master_str]=unstructured
    print(file)
mdata_dct={"prefix":[], "input_text":[], "target_text":[]}
for st,unst in data_dct.items():
    for i in unst:
        mdata_dct['prefix'].append('webNLG')
        mdata_dct['input_text'].append(st)
        mdata_dct['target_text'].append(i)


df1=pd.DataFrame(mdata_dct)
df1.to_csv('webNLG2020.csv')
df1.shape

/content/webnlg-dataset/release_v2.1/xml/train/7triples/7triples_University_train_release.xml
/content/webnlg-dataset/release_v2.1/xml/train/7triples/7triples_Monument_train_release.xml
/content/webnlg-dataset/release_v2.1/xml/train/7triples/7triples_Astronaut_train_release.xml
/content/webnlg-dataset/release_v2.1/xml/train/6triples/6triples_University_train_release.xml
/content/webnlg-dataset/release_v2.1/xml/train/6triples/6triples_Astronaut_train_release.xml
/content/webnlg-dataset/release_v2.1/xml/train/6triples/6triples_Monument_train_release.xml
/content/webnlg-dataset/release_v2.1/xml/train/4triples/4triples_Building_train_release.xml
/content/webnlg-dataset/release_v2.1/xml/train/4triples/4triples_WrittenWork_train_release.xml
/content/webnlg-dataset/release_v2.1/xml/train/4triples/4triples_Artist_train_release.xml
/content/webnlg-dataset/release_v2.1/xml/train/4triples/4triples_CelestialBody_train_release.xml
/content/webnlg-dataset/release_v2.1/xml/train/4triples/4triples_Mea

(44429, 3)

In [ ]:
import glob
import os
import re
import xml.etree.ElementTree as ET

files = glob.glob("/content/dart/data/v1.1.1/dart-v1.1.1-full-train.xml")
triple_re=re.compile('(\d)triples')
data_dct={}
for file in files:
    tree = ET.parse(file)
    root = tree.getroot()
    #triples_num=int(triple_re.findall(file)[0])
    for sub_root in root:
        for ss_root in sub_root:
            strutured_master=[]
            unstructured=[]
            for entry in ss_root:
                unstructured.append(entry.text)
                strutured=[triple.text for triple in entry]
                strutured_master.extend(strutured)
            unstructured=[i for i in unstructured if i.replace('\n','').strip()!='' ]
            strutured_master=strutured_master[-triples_num:]
            strutured_master_str=(' && ').join(strutured_master)
            data_dct[strutured_master_str]=unstructured
    print(file)
mdata_dct={"prefix":[], "input_text":[], "target_text":[]}
for st,unst in data_dct.items():
    for i in unst:
        mdata_dct['prefix'].append('DART')
        mdata_dct['input_text'].append(st)
        mdata_dct['target_text'].append(i)


df2=pd.DataFrame(mdata_dct)
df2.to_csv('DartNLG.csv')
df2.shape

/content/dart/data/v1.1.1/dart-v1.1.1-full-train.xml


(28995, 3)

In [ ]:
df = pd.DataFrame()
df = df.append([df1,df2], ignore_index=True)
df.to_csv('NLGDataset.csv')
df.shape

(73424, 3)

In [ ]:
train_df=pd.read_csv('NLGDataset.csv', index_col=[0])
train_df=train_df.iloc[  :73424,:]
train_df=train_df.sample(frac = 1)
batch_size=8
num_of_batches=int(len(train_df)/batch_size)

In [ ]:
num_of_batches

9178

In [ ]:
train_df.head()

,prefix,input_text,target_text
52303,DART,[TABLECONTEXT] | CLIMATE | Average low °F (°C)...,"In Indo, California, the average low for the y..."
54323,DART,[TABLECONTEXT] | [TITLE] | René Follet && [TAB...,René Follet drew the illustrations of Ivan Zou...
33595,webNLG,Abel_Caballero | inOfficeWhilePrimeMinister | ...,Abel Caballero was in office while Felipe Gonz...
26622,webNLG,"Bakewell_pudding | course | ""Dessert"" && Bakew...",Bakewell pudding is a dessert created in Rutla...
29006,webNLG,"A.S._Roma | ground | ""Rome, Italy"" && A.S._Rom...","A.S. Roma, which has 70634 members, have their..."


In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
    print("Running on the GPU")
else:
    dev = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [ ]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small',
                                             return_dict=True)
#moving the model to GPU
model.to(dev)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [ ]:

optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [ ]:
model.train()
num_of_epochs = 5

loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
  
    running_loss=0

    out = display(progress(1, num_of_batches+1), display_id=True)

    for i in range(num_of_batches):
        inputbatch=[]
        labelbatch=[]
        new_df=train_df[i*batch_size:i*batch_size+batch_size]
        for indx,row in new_df.iterrows():
            input = row['input_text']+'</s>' 
            labels = row['target_text']+'</s>'   
            inputbatch.append(input)
            labelbatch.append(labels)
        inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
        labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
        inputbatch=inputbatch.to(dev)
        labelbatch=labelbatch.to(dev)

        # clear out the gradients of all Variables 
        optimizer.zero_grad()

        # Forward propogation
        outputs = model(input_ids=inputbatch, labels=labelbatch)
        loss = outputs.loss
        loss_num=loss.item()
        logits = outputs.logits
        running_loss+=loss_num
        if i%10 ==0:      
            loss_per_10_steps.append(loss_num)
        out.update(progress(loss_num,i, num_of_batches+1))

        # calculating the gradients
        loss.backward()

        #updating the params
        optimizer.step()
    
    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))


Running epoch: 1


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:562: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  exp_avg_sq_row.mul_(beta2t).add_(1.0 - beta2t, update.mean(dim=-1))


Epoch: 1 , Running loss: 0.7095015854987736
Running epoch: 2


Epoch: 2 , Running loss: 0.5553053921865051
Running epoch: 3


Epoch: 3 , Running loss: 0.49802492654097574
Running epoch: 4


Epoch: 4 , Running loss: 0.45989201657715956
Running epoch: 5


Epoch: 5 , Running loss: 0.43088552397573326


In [ ]:
!mkdir model
torch.save(model.state_dict(),'model/pytorch_model.bin')


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
  
tokenizer = AutoTokenizer.from_pretrained("gagan3012/k2t")

model = AutoModelWithLMHead.from_pretrained("gagan3012/k2t")

device = "cuda:0"
model = model.to(device)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
def generate(keywords, temp = 1.0, top_p = 1.0):
    text = str(keywords)
    text = text.replace(',', '|')
    text = text.replace("'", "")
    text = text.replace('[', '')
    text = text.replace(']', '')
    texts = text.split(".")
    print(text)
    result = ""
    for txt in texts:
        input_ids = tokenizer.encode("{} </s>".format(txt),
                                     return_tensors="pt")
        input_ids = input_ids.to(device)
        outputs = model.generate(input_ids, max_length=1024, temperature=temp, top_p=top_p)
        result += tokenizer.decode(outputs[0])
    result = re.sub('<pad>|</s>', "", result)
    return result

In [ ]:
generate(['India','Life'])

['India| Life']


" India's Life is also the name given to people from India."